In [1]:
import pandas as pd
import numpy as np
import os

In [4]:
df = pd.read_csv("../data/census.csv", delimiter=',', index_col=None)
# df = pd.DataFrame.fr('st1.csv', index_col=None)
df.head(10)

,age,workclass,fnlgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [3]:
print(os.listdir("../data/"))

['census.csv.dvc', '.gitignore', 'census_cleaned.csv.dvc', 'census.csv', 'census_cleaned_test.csv', 'census_cleaned.csv']


In [103]:
print('# File sizes')
for f in os.listdir('../data'):
    print(f.ljust(30) + str(round(os.path.getsize('../data/' + f) / 1000000, 2)) + 'MB') 

# File sizes
lb_census.joblib              0.0MB
census_model_classifier.joblib0.0MB
encoder_census.joblib         0.0MB
census.csv.dvc                0.0MB
.gitignore                    0.0MB
census_cleaned.csv.dvc        0.0MB
census.csv                    3.97MB
census_cleaned_test.csv       0.66MB
census_cleaned.csv            3.3MB


In [4]:
# observe the imbalance between the category of the two salaries
df[' salary'].value_counts(normalize = True)

 <=50K    0.75919
 >50K     0.24081
Name:  salary, dtype: float64

The Summary shows that there is no missing values. However we can observe that there is some ? here and there
- One can observe that we have some "?" for some columns.
- Let's first define our categorical and numerical features 

In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              32561 non-null  int64 
 1    workclass       32561 non-null  object
 2    fnlgt           32561 non-null  int64 
 3    education       32561 non-null  object
 4    education-num   32561 non-null  int64 
 5    marital-status  32561 non-null  object
 6    occupation      32561 non-null  object
 7    relationship    32561 non-null  object
 8    race            32561 non-null  object
 9    sex             32561 non-null  object
 10   capital-gain    32561 non-null  int64 
 11   capital-loss    32561 non-null  int64 
 12   hours-per-week  32561 non-null  int64 
 13   native-country  32561 non-null  object
 14   salary          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [10]:
# Let's see the categorical features 
df_categorical = df.select_dtypes(exclude=[np.number])
df_cat_val = df_categorical.columns.values
print(df_cat_val)

['workclass' 'education' 'marital-status' 'occupation' 'relationship'
 'race' 'sex' 'native-country' 'salary']


In [11]:
# Let's see the numerical features 
df_numeric = df.select_dtypes(include=[np.number])
df_numeric_val = df_numeric.columns.values
print(df_numeric_val)

['age' 'fnlgt' 'education-num' 'capital-gain' 'capital-loss'
 'hours-per-week']


In [12]:
# Let's see what each categorical column features takes as a value
for col in df_cat_val:
    print(f"{col}: {df[col].unique()}",'\n')

workclass: [' State-gov' ' Self-emp-not-inc' ' Private' ' Federal-gov' ' Local-gov'
 ' ?' ' Self-emp-inc' ' Without-pay' ' Never-worked'] 

education: [' Bachelors' ' HS-grad' ' 11th' ' Masters' ' 9th' ' Some-college'
 ' Assoc-acdm' ' Assoc-voc' ' 7th-8th' ' Doctorate' ' Prof-school'
 ' 5th-6th' ' 10th' ' 1st-4th' ' Preschool' ' 12th'] 

marital-status: [' Never-married' ' Married-civ-spouse' ' Divorced'
 ' Married-spouse-absent' ' Separated' ' Married-AF-spouse' ' Widowed'] 

occupation: [' Adm-clerical' ' Exec-managerial' ' Handlers-cleaners' ' Prof-specialty'
 ' Other-service' ' Sales' ' Craft-repair' ' Transport-moving'
 ' Farming-fishing' ' Machine-op-inspct' ' Tech-support' ' ?'
 ' Protective-serv' ' Armed-Forces' ' Priv-house-serv'] 

relationship: [' Not-in-family' ' Husband' ' Wife' ' Own-child' ' Unmarried'
 ' Other-relative'] 

race: [' White' ' Black' ' Asian-Pac-Islander' ' Amer-Indian-Eskimo' ' Other'] 

sex: [' Male' ' Female'] 

native-country: [' United-States' ' Cuba'

## Important notice the following 
* The presence of spaces that can generate many erros.
* "`workclass`" and "`occupation`" contains interogation mark with an initial space
* We need first to eliminate those spaces 

## Clean categorical columns names


In [21]:
print(df.columns)
df.columns = df.columns.str.replace(' ','')
print(df.columns)

Index(['age', 'workclass', 'fnlgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'salary'],
      dtype='object')
Index(['age', 'workclass', 'fnlgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'salary'],
      dtype='object')


## Clean categorical columns

In [14]:
# Initialize our categorical names with the cleaned names 
# The categorical features 
df_categorical = df.select_dtypes(exclude=[np.number])
df_cat_val = df_categorical.columns.values
print(df_cat_val)

['workclass' 'education' 'marital-status' 'occupation' 'relationship'
 'race' 'sex' 'native-country' 'salary']


In [18]:
df_categorical.columns

Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country', 'salary'],
      dtype='object')

In [23]:
# for col in df_cat_val:
#     df[col] = df[col].str.replace(' ', '')
    
# Another way with the lamdba function 
df[df_categorical.columns]=  df_categorical.apply(lambda x: x.str.strip()) 

In [24]:
for col in df_cat_val:
    print(f"{col}: {df[col].unique()}",'\n')

workclass: ['State-gov' 'Self-emp-not-inc' 'Private' 'Federal-gov' 'Local-gov' '?'
 'Self-emp-inc' 'Without-pay' 'Never-worked'] 

education: ['Bachelors' 'HS-grad' '11th' 'Masters' '9th' 'Some-college' 'Assoc-acdm'
 'Assoc-voc' '7th-8th' 'Doctorate' 'Prof-school' '5th-6th' '10th'
 '1st-4th' 'Preschool' '12th'] 

marital-status: ['Never-married' 'Married-civ-spouse' 'Divorced' 'Married-spouse-absent'
 'Separated' 'Married-AF-spouse' 'Widowed'] 

occupation: ['Adm-clerical' 'Exec-managerial' 'Handlers-cleaners' 'Prof-specialty'
 'Other-service' 'Sales' 'Craft-repair' 'Transport-moving'
 'Farming-fishing' 'Machine-op-inspct' 'Tech-support' '?'
 'Protective-serv' 'Armed-Forces' 'Priv-house-serv'] 

relationship: ['Not-in-family' 'Husband' 'Wife' 'Own-child' 'Unmarried' 'Other-relative'] 

race: ['White' 'Black' 'Asian-Pac-Islander' 'Amer-Indian-Eskimo' 'Other'] 

sex: ['Male' 'Female'] 

native-country: ['United-States' 'Cuba' 'Jamaica' 'India' '?' 'Mexico' 'South'
 'Puerto-Rico' 'Hondura

## Remark-1
* Note that we have eliminated all white spaces in all categorical features 
* Let's replace all "?" in categorical features with "NaN"

In [113]:
df[ df =='?'] = np.nan

In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       30725 non-null  object
 2   fnlgt           32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      30718 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  31978 non-null  object
 14  salary          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


## Remark-2
* workclass, occupation and native-country used to have "?"
* workclass, occupation and native-country now contains "NaN" values. 

In [115]:
df.isna().sum()

age                  0
workclass         1836
fnlgt                0
education            0
education-num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
salary               0
dtype: int64

In [116]:
df['occupation'].mode()

0    Prof-specialty
dtype: object

## Clean data
* Either use mode to replace missing values with the most common element in the column feature
* Or completely delete the data row including the missing value
* I will initially choose the second option, that is to delete the row with missing values

In [117]:
# # Impute categorical missing values 
# for col in ['workclass', 'occupation', 'native-country']:
#     df[col].fillna(df[col].mode()[0], inplace=True)


In [118]:
# Delete categorical missing values 
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30162 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             30162 non-null  int64 
 1   workclass       30162 non-null  object
 2   fnlgt           30162 non-null  int64 
 3   education       30162 non-null  object
 4   education-num   30162 non-null  int64 
 5   marital-status  30162 non-null  object
 6   occupation      30162 non-null  object
 7   relationship    30162 non-null  object
 8   race            30162 non-null  object
 9   sex             30162 non-null  object
 10  capital-gain    30162 non-null  int64 
 11  capital-loss    30162 non-null  int64 
 12  hours-per-week  30162 non-null  int64 
 13  native-country  30162 non-null  object
 14  salary          30162 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


# Save cleaned dataset


In [73]:
# df.to_csv("../data/census_cleaned.csv", index=False)

In [75]:
required_columns = [
        'age',
        'workclass',
        'fnlgt',
        'education',
        'education-num',
        'marital-status',
        'occupation',
        'relationship',
        'race',
        'sex',
        'capital-gain',
        'capital-loss',
        'hours-per-week',
        'native-country',
        'salary'
    ]

In [119]:
df.columns.values

array(['age', 'workclass', 'fnlgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'salary'], dtype=object)

In [120]:
set(df.columns.values)

{'age',
 'capital-gain',
 'capital-loss',
 'education',
 'education-num',
 'fnlgt',
 'hours-per-week',
 'marital-status',
 'native-country',
 'occupation',
 'race',
 'relationship',
 'salary',
 'sex',
 'workclass'}

In [80]:
set(required_columns)

{'age',
 'capital-gain',
 'capital-loss',
 'education',
 'education-num',
 'fnlgt',
 'hours-per-week',
 'marital-status',
 'native-country',
 'occupation',
 'race',
 'relationship',
 'salary',
 'sex',
 'workclass'}

In [123]:
assert set(df.columns.values).issuperset(set(required_columns))

In [121]:
assert set(df.columns.values)==set(required_columns)

In [ ]:
assert set(data.columns.values)==(set(required_columns.keys()))

    for col_name, format_verification_funct in required_columns.items():
        assert format_verification_funct(data[col_name]), f"Column {col_name} failed test {format_verification_funct}"